In [1]:
epochs, batch, lr = 100, 64, 1e-4
num_clients, in_channels, num_classes = 6, 3, 10
Dataset, Model, Prefix = 'CIFAR', 'VGG_11', 'Train1'
Gdrive = '/content/drive/MyDrive/Colab Notebooks/SplitLearning3'
import torch; torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os; os.chdir(Gdrive)
%run library/Dataset.ipynb
%run library/Network.ipynb
%run library/Training.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
dataset, testset = loadDataset()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /content/cifar-10-python.tar.gz to /content/
Files already downloaded and verified


In [16]:
class SplitNN:
  def __init__(self, clients, server):
    self.clients = [c.cuda() for c in clients]
    self.server = server.cuda()

  def initialize(self, datasets, tester, epochs, batch, lr):
    for client, data in zip(self.clients, datasets):
      client.initialize(data, epochs, batch, lr)
    self.server.initialize(tester, epochs, lr)
    self.cached = [[] for c in self.clients]

  def evaluate(self):
    with torch.no_grad():
      self.server.eval()
      for client in self.clients:
        client.eval()
        client.accuracy.append(0)
        for images, labels in self.server.loader:
          output = self.server(client(images)).argmax(1)
          client.accuracy[-1] += (output == labels).sum().item()
        client.accuracy[-1] /= len(self.server.loader.dataset)
  
  def train_network(self, epoch, private=True, sequence=False, federate=False, caches=None, shuffle=True):
    local = lambda m, p: m.client[0] if p else m
    self.server.train()
    if shuffle:
      index = torch.randperm(len(self.clients))
    else: index = torch.arange(len(self.clients))
    clients = [self.clients[i] for i in index]
    if sequence: #SL
      model = local(clients[-1], private).state_dict()
    for idx, client in enumerate(clients):
      if caches: #Our
        index2 = index.tolist(); index2.remove(idx)
        cached = [self.cached[i] for i in index]
        maxlen = [caches[i] for i in index]
      if sequence: #SL
        local(client, private).load_state_dict(model)
      client.train()
      for images, labels in client.loader:
        client.zero_grad(), self.server.zero_grad()
        #output = self.server(client(images))
        output = client(images)
        #always cache data
        if len(labels) == batch:
          self.cached[idx] = (output.clone(), labels)
        else:
          self.cached[idx] = (torch.cat([self.cached[idx][0], output.clone()]),
                              torch.cat([self.cached[idx][1], labels]))
        if caches and epoch > 0:
          index3 = [torch.randperm(len(c)) for _,c in cached]
          images_cached = [c[0][i][:m] for c,i,m in zip(cached,index3,maxlen)]
          labels_cached = [c[1][i][:m] for c,i,m in zip(cached,index3,maxlen)]
          output = torch.cat([output] + images_cached)
          labels = torch.cat([labels] + labels_cached)
        output = self.server(output)
        loss = F.cross_entropy(output, labels)
        grads = self.server.backward(loss)
        client.backward(grads[:len(client.output)])
        #client.backward(self.server.backward(loss))
        client.optim_step(), self.server.optim_step()
      client.sched_step()
      if sequence: #SL
        model = local(client, private).state_dict()
    self.server.sched_step()
    if federate: #SFL
      models = [local(c, private).state_dict() for c in clients]
      for key in models[0]:
        for i in range(1, len(models)):
          models[0][key] += models[i][key]
        if models[0][key].type().split('.')[-1] == 'LongTensor':
          torch.div(models[0][key], len(models), rounding_mode='floor')
        else: models[0][key] /= len(models)
      for client in clients:
        local(client, private).load_state_dict(models[0])

In [ ]:
# private training
train_splitnn(clients, (testset), 'private', 60)

  0%|          | 0/60 [00:00<?, ?it/s]

Output()

In [ ]:
train_splitnn(clients, (testset), 'sequence')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [ ]:
caches = [8 for i in range(num_clients)]
train_splitnn(clients, (testset), 'private', caches=caches)

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

In [6]:
clients = torch.ones((num_clients,))
clients = ratio_split(dataset, clients / clients.sum())

In [17]:
train_splitnn(clients, (testset), 'private', 60)

  0%|          | 0/60 [00:00<?, ?it/s]

Output()

#Imbalance

In [ ]:
clients = np.array([1, 3, 9, 19, 30, 38])
clients = ratio_split(dataset, clients / clients.sum())

In [ ]:
# separated training
saveAccuracies(Gdrive, [m.clients[0] for m in train_separate(clients, (testset))], 'ImB-Non')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Non.acc


In [ ]:
# private training
saveAccuracies(Gdrive, train_splitnn(clients, (testset), 'private').clients, 'ImB-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Prv.acc


In [ ]:
# sequential training privately
saveAccuracies(Gdrive, train_splitnn(clients, (testset), 'sequence_private').clients, 'ImB-Seq-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Seq-Prv.acc


In [ ]:
# federated training privately
saveAccuracies(Gdrive, train_splitnn(clients, (testset), 'federate_private').clients, 'ImB-Fed-Prv')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Fed-Prv.acc


In [ ]:
# sequential training
saveAccuracies(Gdrive, train_splitnn(clients, (testset), 'sequence').clients, 'ImB-Seq')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Seq.acc


In [ ]:
# federated training
saveAccuracies(Gdrive, train_splitnn(clients, (testset), 'federate').clients, 'ImB-Fed')

  0%|          | 0/100 [00:00<?, ?it/s]

Output()

/content/drive/MyDrive/Colab Notebooks/SplitLearning3/data/CIFAR-VGG_11-Train1-6-ImB-Fed.acc
